# Cassandra

Из докера: https://hub.docker.com/_/cassandra

In [ ]:
cqlsh> DROP KEYSPACE IF EXISTS lesson6;
cqlsh> CREATE KEYSPACE IF NOT EXISTS lesson6 WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor': 1 };
cqlsh> USE lesson6;
cqlsh:lesson6> CREATE TABLE IF NOT EXISTS db (
           ...     id int, 
           ...     name text,
           ...     size text,
           ...     primary key (id)
           ... );
cqlsh:lesson6> INSERT INTO db (id, name, size) VALUES (3, 'Deer', 'Big');
cqlsh:lesson6> SELECT * FROM db;

 id | name | size
----+------+------
  3 | Deer |  Big

(1 rows)
cqlsh:lesson6> INSERT INTO db (id, name) VALUES (3, 'Doe');
cqlsh:lesson6> SELECT * FROM db;

 id | name | size
----+------+------
  3 |  Doe |  Big

(1 rows)
cqlsh:lesson6> INSERT INTO db (id, name) VALUES (5, 'Snake');
cqlsh:lesson6> SELECT * FROM db;

 id | name  | size
----+-------+------
  5 | Snake | null
  3 |   Doe |  Big

(2 rows)
cqlsh:lesson6> DELETE id FROM db where id = 3; -- вызывает ошибку - нельзя удалять в консоли по ключу. Только по значению
InvalidRequest: Error from server: code=2200 [Invalid query] message="Invalid identifier id for deletion (should not be a PRIMARY KEY part)"
cqlsh:lesson6> INSERT INTO db (id, name, size) VALUES (3, null, null);
cqlsh:lesson6> SELECT * FROM db;

 id | name  | size
----+-------+------
  5 | Snake | null
  3 |  null | null

(2 rows)
cqlsh:lesson6> SELECT count(*) FROM db;

 count
-------
     2

(1 rows)

Warnings :
Aggregation query used without partition key

cqlsh:lesson6> drop table db;
cqlsh:lesson6> SELECT * FROM db; -- вызывает ошибку - нет такой таблицы
InvalidRequest: Error from server: code=2200 [Invalid query] message="table db does not exist"

In [5]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,com.datastax.spark:spark-cassandra-connector_2.12:3.2.0 pyspark-shell'

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType, IntegerType
from pyspark.sql.functions import from_json, to_json, col, struct
import time
import signal
from typing import Union

from tools_kafka import Kafka
from tools_pyspark_hdfs import Spark_HDFS as HDFS
from tools_pyspark import stop_all_streams, sink, read_stream_kafka, console_stream, console_clear, console_show

spark = SparkSession.builder.appName("my_spark").getOrCreate()

cass_animals_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="db", keyspace="lesson6") \
    .load()

cass_animals_df.printSchema()
cass_animals_df.show()

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- size: string (nullable = true)

+---+-----+----+
| id| name|size|
+---+-----+----+
|  5|Snake|null|
|  3| Deer| Big|
|  4|  Doe|null|
+---+-----+----+



In [7]:
# добавим строку в таблицу
# для этого сначала создадим датафрейм
cow_df = spark.sql("""select 11 as id, "Cow" as name, "Big" as size """)
cow_df.show()

# запишем фрейм в таблицу
cow_df.write \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="db", keyspace="lesson6") \
    .mode("append") \
    .save()

cass_animals_df.show()

+---+----+----+
| id|name|size|
+---+----+----+
| 11| Cow| Big|
+---+----+----+

+---+-----+----+
| id| name|size|
+---+-----+----+
|  5|Snake|null|
| 11|  Cow| Big|
|  3| Deer| Big|
|  4|  Doe|null|
+---+-----+----+



In [8]:
cass_big_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="db", keyspace="lesson6") \
    .load()

cass_big_df.filter(F.col("name")=="Cow").show()

+---+----+----+
| id|name|size|
+---+----+----+
| 11| Cow| Big|
+---+----+----+



In [14]:
# Проверим пушит ли спарк фильтры в касандру.
def explain(self, extended=True):
    if extended:
        print(self._jdf.queryExecution().toString())
    else:
        print(self._jdf.queryExecution().simpleString())

cass_big_df.filter(F.col("id")==11).explain()

== Physical Plan ==
*(1) Project [id#107, name#108, size#109]
+- BatchScan[id#107, name#108, size#109] Cassandra Scan: lesson6.db
 - Cassandra Filters: [["id" = ?, 11]]
 - Requested Columns: [id,name,size] RuntimeFilters: []




In [21]:
# Сделаем представление и используем его в выражении
cass_big_df.createOrReplaceTempView("cass_df")

spark.sql("""
SELECT * 
FROM cass_df
WHERE id BETWEEN 6 and 12
""").explain()

== Physical Plan ==
*(1) Project [id#107, name#108, size#109]
+- *(1) Filter ((id#107 >= 6) AND (id#107 <= 12))
   +- BatchScan[id#107, name#108, size#109] Cassandra Scan: lesson6.db
 - Cassandra Filters: []
 - Requested Columns: [id,name,size] RuntimeFilters: []




In [23]:
# Как видно из результата, фильтрацию будет делать не кассандра, а спарк, что медленно.
# Перепишем запрос, чтобы запрос фильтровался на стороне базы
spark.sql("""
SELECT * 
FROM cass_df
WHERE id IN (4, 11)
""").explain()

== Physical Plan ==
*(1) Project [id#107, name#108, size#109]
+- BatchScan[id#107, name#108, size#109] Cassandra Scan: lesson6.db
 - Cassandra Filters: [["id" IN (?, ?), (4, 11)]]
 - Requested Columns: [id,name,size] RuntimeFilters: []


